This is based on Ole's idea . The guy is genius.

https://www.kaggle.com/wikunia

https://opensourc.es/blog/kaggle-santa-2019's 

also thanks to Vlamidir for idea to use lapsolver . 

I will add more details in notebook . This is a starter and based on below idea from Ole. It is detailed in his blog.

**Okay now my idea was that we simply put 50 families into each day and hope that it fits the hard constraint of 125≤Nd≤300. Most families have 4 people which you can also see in the Santa finances notebook so I guessed it might work.
****
This is of course just a rough estimation but if 50 families would be correct for each day this would be optimal for the preference cost and we shouldn't be too bad for the accounting as the fluctuation between two days should be okay.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm_notebook as tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Read in the family information and sample submission

In [ ]:
fpath = '/kaggle/input/santa-2019-workshop-scheduling/family_data.csv'
data = pd.read_csv(fpath)

# fpath = '/kaggle/input/santa-2019-workshop-scheduling/sample_submission.csv'
# submission = pd.read_csv(fpath, index_col='family_id')

# thanks for https://www.kaggle.com/cweed28/reduce-memory-usage to reduce memory

data['family_id'] = data['family_id'].astype('uint16')
data[data.drop('family_id',axis=1).columns] = data[data.drop('family_id',axis=1).columns].astype('uint8')

new_memory = data.memory_usage().sum() / (1024**2 )
print(f'{new_memory:.4f} MB ')

In [ ]:
fpath = '/kaggle/input/santa-2019-workshop-scheduling/sample_submission.csv'
submission = pd.read_csv(fpath)

In [ ]:
data.head()

In [ ]:
submission.head()

In [ ]:
! pip install munkres

In [ ]:
from munkres import Munkres

In [ ]:
def get_choice_cost(choice, n):
    if choice == 0:
        return int(0)
    elif choice == 1:
        return int(50)
    elif choice == 2:
        return int(50+9*n)
    elif choice == 3:
        return int(100+9*n)
    elif choice == 4:
        return int(200+9*n)
    elif choice == 5:
        return int(200+18*n)
    elif choice == 6:
        return int(300+18*n)
    elif choice == 7:
        return int(300+36*n)
    elif choice == 8:
        return int(400+36*n)
    elif choice == 9:
        return int(500+(36+199)*n)
    else:
        return int(500+434*n)

In [ ]:
def get_weights():
    #dff = CSV.read("family_data.csv")
    

    # preference cost matrix
    preference_mat = np.zeros((5000,5000))
    preference_mat = preference_mat.astype(int)
    # first fill every column with the maximum cost for that family
    for ind,row in data.iterrows():
        preference_mat[ind] = get_choice_cost(10, row['n_people'])
    for ind,row in data.iterrows():
        for i in range(10):
            
            
            choice =  row[f'choice_{i}']
            choice_cost = get_choice_cost(i, row['n_people'])
            for k in range(((choice-1)*50),(choice)*50):
                preference_mat[ind,k] = choice_cost
    preference_mat = preference_mat.astype(int)
    return preference_mat
                
                    
                    




In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
# from cachetools import cached, TTLCache  # 1 - let's import the "cached" decorator and the "TTLCache" object from cachetools
# cache = TTLCache(maxsize=100, ttl=300)

In [ ]:
#weights = get_weights()

In [ ]:
weights = get_weights()


In [ ]:
# weights = weights[:200,:200]
weights = weights.astype('uint16')

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
!pip install lapsolver

In [ ]:
from lapsolver import solve_dense
rids, cids = solve_dense(weights)

In [ ]:
submission['assigned_day'] = np.ceil(cids/50)

In [ ]:
# one of the row has 0 ; lets make it 100 since 100 days has 49 entries

ind1 = submission[submission['assigned_day'] == 0].index[0]


In [ ]:
ind1

In [ ]:
submission.columns

In [ ]:
submission['assigned_day'].value_counts()

In [ ]:
submission.loc[3195,'assigned_day']

In [ ]:
submission.loc[ind1,'assigned_day']= 100

In [ ]:
submission['assigned_day'].value_counts()

In [ ]:
submission['assigned_day'] = submission['assigned_day'].astype(int)

In [ ]:
submission['assigned_day'].value_counts()

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission_hungarian.csv",index = False)

In [ ]:

# from munkres import Munkres, print_matrix
# m = Munkres()
# #def main_run():
# # weights = get_weights()
# # weights = weights[:10,:10]

# #assignment, cost = hungarian(weights)
# indexes = m.compute(weights)
# #print_matrix(weights, msg='Lowest cost through this matrix:')
# total = 0
# # for row, column in indexes:
# #     value = weights[row][column]
# #     total += value
# #     #print(f'({row}, {column}) -> {value}')
# # print(f'total cost: {total}')

In [ ]:
# family_size_dict = data[['n_people']].to_dict()['n_people']

# cols = [f'choice_{i}' for i in range(10)]
# choice_dict = data[cols].T.to_dict()

# N_DAYS = 100
# MAX_OCCUPANCY = 300
# MIN_OCCUPANCY = 125

# # from 100 to 1
# days = list(range(N_DAYS,0,-1))

In [ ]:
# family_size_ls = list(family_size_dict.values())
# choice_dict_num = [{vv:i for i, vv in enumerate(di.values())} for di in choice_dict.values()]

# # Computer penalities in a list
# penalties_dict = {
#     n: [
#         0,
#         50,
#         50 + 9 * n,
#         100 + 9 * n,
#         200 + 9 * n,
#         200 + 18 * n,
#         300 + 18 * n,
#         300 + 36 * n,
#         400 + 36 * n,
#         500 + 36 * n + 199 * n,
#         500 + 36 * n + 398 * n
#     ]
#     for n in range(max(family_size_dict.values())+1)
# } 

In [ ]:
# def cost_function(prediction):
#     penalty = 0

#     # We'll use this to count the number of people scheduled each day
#     daily_occupancy = {k:0 for k in days}
    
#     # Looping over each family; d is the day, n is size of that family, 
#     # and choice is their top choices
#     for n, d, choice in zip(family_size_ls, prediction, choice_dict_num):
#         # add the family member count to the daily occupancy
#         daily_occupancy[d] += n

#         # Calculate the penalty for not getting top preference
#         if d not in choice:
#             penalty += penalties_dict[n][-1]
#         else:
#             penalty += penalties_dict[n][choice[d]]

#     # for each date, check total occupancy
#     #  (using soft constraints instead of hard constraints)
#     for v in daily_occupancy.values():
#         if (v > MAX_OCCUPANCY) or (v < MIN_OCCUPANCY):
#             penalty += 100000000

#     # Calculate the accounting cost
#     # The first day (day 100) is treated special
#     accounting_cost = (daily_occupancy[days[0]]-125.0) / 400.0 * daily_occupancy[days[0]]**(0.5)
#     # using the max function because the soft constraints might allow occupancy to dip below 125
#     accounting_cost = max(0, accounting_cost)
    
#     # Loop over the rest of the days, keeping track of previous count
#     yesterday_count = daily_occupancy[days[0]]
#     for day in days[1:]:
#         today_count = daily_occupancy[day]
#         diff = abs(today_count - yesterday_count)
#         accounting_cost += max(0, (daily_occupancy[day]-125.0) / 400.0 * daily_occupancy[day]**(0.5 + diff / 50.0))
#         yesterday_count = today_count

#     penalty += accounting_cost

#     return penalty

In [ ]:
# # Start with the sample submission values
# best = submission['assigned_day'].tolist()
# start_score = cost_function(best)


# new = best.copy()
# # loop over each family
# for fam_id, _ in enumerate(tqdm(best)):
#     # loop over each family choice
#     for pick in range(10):
#         day = choice_dict[fam_id][f'choice_{pick}']
#         temp = new.copy()
#         temp[fam_id] = day # add in the new pick
#         if cost_function(temp) < start_score:
#             new = temp.copy()
#             start_score = cost_function(new)

# submission['assigned_day'] = new
# score = cost_function(new)
# submission.to_csv(f'submission_{score}.csv')
# print(f'Score: {score}')

I will update further based on his ideas